In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dis
import time
import threading

from circular_queue.unsafe import UnsafeCircularQueue
from dinning_philosopher.fork import Fork
from dinning_philosopher.philosopher import DeadLockPhilosopher
from dinning_philosopher.philosopher import ServedPhilosopher

# Race Conditions

In [3]:
def unsafe_increment(total):
    global n
    for i in range(total):
        n += 1

In [4]:
n = 0
unsafe_increment(1000000)
unsafe_increment(1000000)
print(n)

2000000


In [5]:
n = 0
thread1 = threading.Thread(target=unsafe_increment, args=(1000000,))
thread2 = threading.Thread(target=unsafe_increment, args=(1000000,))

thread1.start()
thread2.start()
thread1.join()
thread2.join()
print(n)

1545276


`n += 1` 字节码：

In [6]:
import dis
def foo():
    global n
    n += 1
dis.dis(foo, depth=0)

  4           0 LOAD_GLOBAL              0 (n)
              2 LOAD_CONST               1 (1)
              4 INPLACE_ADD
              6 STORE_GLOBAL             0 (n)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


# 锁

In [7]:
def safe_increment(total, lock):
    global n
    for i in range(total):
        with lock:
            n += 1


n = 0
lock = threading.Lock()
thread1 = threading.Thread(target=safe_increment, args=(1000000, lock))
thread2 = threading.Thread(target=safe_increment, args=(1000000, lock))

thread1.start()
thread2.start()
thread1.join()
thread2.join()
print(n)

2000000


## 死锁

### 生产者/消费者

In [8]:
def consume(queue, total):
    for i in range(total):
        if not queue.is_empty():
            print(i, queue.remove())
            time.sleep(0.1)

def produce(queue, total, message):
    for i in range(total):
        if not queue.is_full():
            queue.add('{:d}: {:s}'.format(i, message))
            time.sleep(0.1)

### 线程不安全的队列

In [9]:
unsafe_queue = UnsafeCircularQueue(10)

thread1 = threading.Thread(target=produce, args=(unsafe_queue, 100, "Hello!"))
thread2 = threading.Thread(target=produce, args=(unsafe_queue, 100, "Goodbye!"))
thread3 = threading.Thread(target=consume, args=(unsafe_queue, 200))

thread1.start()
thread2.start()
thread3.start()
thread1.join()
thread2.join()
thread3.join()

0 0: Hello!
1 0: Goodbye!
2 1: Hello!
3 1: Goodbye!
4 2: Hello!
5 2: Goodbye!
6 3: Hello!
7 3: Goodbye!
8 4: Hello!
9 4: Goodbye!
10 5: Hello!
11 5: Goodbye!
12 6: Goodbye!
13 6: Hello!
14 7: Goodbye!
15 7: Hello!
16 8: Hello!
17 8: Goodbye!
18 9: Hello!
19 10: Hello!
20 11: Hello!
21 12: Hello!
22 13: Hello!
23 14: Hello!
24 15: Hello!
25 16: Hello!
26 17: Hello!
27 18: Hello!
28 19: Hello!
29 20: Hello!
30 21: Hello!
31 22: Hello!
32 23: Hello!
33 24: Hello!
34 25: Hello!
35 26: Hello!
36 27: Hello!
37 28: Hello!
38 29: Hello!
39 30: Hello!
40 31: Hello!
41 32: Hello!
42 33: Hello!
43 34: Hello!
44 35: Hello!
45 36: Hello!
46 37: Hello!
47 38: Hello!
48 39: Hello!
49 40: Hello!
50 41: Hello!
51 42: Hello!
52 43: Hello!
53 44: Hello!
54 45: Hello!
55 46: Hello!
56 47: Hello!
57 48: Hello!
58 49: Hello!
59 50: Hello!
60 51: Hello!
61 52: Hello!
62 53: Hello!
63 54: Hello!
64 55: Hello!
65 56: Hello!
66 57: Hello!
67 58: Hello!
68 59: Hello!
69 60: Hello!
70 61: Hello!
71 62: Hello!
72 

### 死锁队列

# 应用：哲学家就餐问题

In [10]:
def generate_forks(n):
    forks = list()
    for i in range(n):
        forks.append(Fork(i))
    return forks


def arrange_philosophers(philosophers, forks):
    assert len(philosophers) == len(forks)
    
    n = len(philosophers)
    for i in range(n):
        left_fork = forks[(i + 1) % n]
        right_fork = forks[i]
        philosophers[i].set_left_fork(left_fork)
        philosophers[i].set_right_fork(right_fork)
        print(philosophers[i])


def dinning_philosophers_main(philosophers):
    try:
        threads = list()
        for phil in philosophers:
            t = threading.Thread(target=phil)
            threads.append(t)
            t.start()
        for t in threads:
            t.join()
    except KeyboardInterrupt:
        for t in threads:
            t.is_alive = False

NUM_PHILOSOPHERS = 5

## 死锁解法

In [11]:
forks = generate_forks(NUM_PHILOSOPHERS)
philosophers = list()
for i in range(NUM_PHILOSOPHERS):
    philosophers.append(DeadLockPhilosopher(i))
arrange_philosophers(philosophers, forks)

DeadLockPhilosopher[index=0, left_fork=Fork[item_index=1], right_fork=Fork[item_index=0]]
DeadLockPhilosopher[index=1, left_fork=Fork[item_index=2], right_fork=Fork[item_index=1]]
DeadLockPhilosopher[index=2, left_fork=Fork[item_index=3], right_fork=Fork[item_index=2]]
DeadLockPhilosopher[index=3, left_fork=Fork[item_index=4], right_fork=Fork[item_index=3]]
DeadLockPhilosopher[index=4, left_fork=Fork[item_index=0], right_fork=Fork[item_index=4]]


下一行代码会由于死锁完全阻塞住，直接终止下一行代码的运行即可。

In [12]:
dinning_philosophers_main(philosophers)

Philosopher #0 is taking fork #1.Philosopher #1 is taking fork #2.
Philosopher #2 is taking fork #3.Philosopher #3 is taking fork #4.


Philosopher #4 is taking fork #0.


## 服务生法

In [14]:
waiter = threading.Lock()
forks = generate_forks(NUM_PHILOSOPHERS)
philosophers = list()
for i in range(NUM_PHILOSOPHERS):
    philosophers.append(ServedPhilosopher(i, waiter))
arrange_philosophers(philosophers, forks)

ServedPhilosopher[index=0, left_fork=Fork[item_index=1], right_fork=Fork[item_index=0]][waiter=<unlocked _thread.lock object at 0x7ff4782627b0>]
ServedPhilosopher[index=1, left_fork=Fork[item_index=2], right_fork=Fork[item_index=1]][waiter=<unlocked _thread.lock object at 0x7ff4782627b0>]
ServedPhilosopher[index=2, left_fork=Fork[item_index=3], right_fork=Fork[item_index=2]][waiter=<unlocked _thread.lock object at 0x7ff4782627b0>]
ServedPhilosopher[index=3, left_fork=Fork[item_index=4], right_fork=Fork[item_index=3]][waiter=<unlocked _thread.lock object at 0x7ff4782627b0>]
ServedPhilosopher[index=4, left_fork=Fork[item_index=0], right_fork=Fork[item_index=4]][waiter=<unlocked _thread.lock object at 0x7ff4782627b0>]


In [13]:
dinning_philosophers_main(philosophers)

Philosopher #0 is taking fork #1.
Philosopher #0 is taking fork #0.
Philosopher #0 is eating.(1 totally)Philosopher #1 is taking fork #2.

Philosopher #0 is putting down fork #1.
Philosopher #0 is putting down fork #0.
Philosopher #0 is thinking.(1 totally)
Philosopher #1 is taking fork #1.
Philosopher #1 is eating.(1 totally)Philosopher #2 is taking fork #3.

Philosopher #1 is putting down fork #2.
Philosopher #1 is putting down fork #1.
Philosopher #1 is thinking.(1 totally)
Philosopher #2 is taking fork #2.
Philosopher #2 is eating.(1 totally)Philosopher #3 is taking fork #4.

Philosopher #2 is putting down fork #3.
Philosopher #2 is putting down fork #2.
Philosopher #2 is thinking.(1 totally)
Philosopher #3 is taking fork #3.
Philosopher #3 is eating.(1 totally)Philosopher #4 is taking fork #0.

Philosopher #3 is putting down fork #4.
Philosopher #3 is putting down fork #3.
Philosopher #3 is thinking.(1 totally)
Philosopher #4 is taking fork #4.
Philosopher #4 is eating.(1 totally)